## **Installing Dependencies**

In [ ]:
!pip install datasets bitsandbytes torch transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `thesis` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `thesis`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## **Importing Libraries**

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import random
import numpy as np
from collections import Counter
import torch
import re
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

## **Load the data**

In [ ]:
hallu_factalign = load_dataset('chaoweihuang/factalign-gemma2-f1_0.75', trust_remote_code=True)

README.md:   0%|          | 0.00/839 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/738k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2177 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/385 [00:00<?, ? examples/s]

## **Data Preparation**

In [ ]:
# Extraction of the prompt and the response from the JSON format

def get_question_answer(example):
    return {
        "prompt": example["prompt"][0]["content"],
        "completion": example["completion"][0]["content"]
    }

hallu_factalign = hallu_factalign.map(get_question_answer)

Map:   0%|          | 0/2177 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

In [ ]:
# Convert to Pandas Dataframe for better handling
hallu_factalign_train = Dataset.to_pandas(hallu_factalign['train'])
hallu_factalign_test = Dataset.to_pandas(hallu_factalign['test'])

In [ ]:
# Convert the boolean values True or False in integers
hallu_factalign_train['label'] = hallu_factalign_train['label'].apply(lambda x: int(x))
hallu_factalign_test['label'] = hallu_factalign_test['label'].apply(lambda x: int(x))

In [ ]:
# Remove the instances without response
hallu_factalign_train = hallu_factalign_train[~hallu_factalign_train['completion'].isna()].reset_index(drop=True)
hallu_factalign_test = hallu_factalign_test[~hallu_factalign_train['completion'].isna()].reset_index(drop=True)

In [ ]:
# Remove the suffix "Provide as many specific details and examples as possible (such as names of people, numbers, events, locations, dates, times, etc.)"
def extract_question(prompt):
    return prompt.split("?")[0] + "?" if "?" in prompt else prompt

hallu_factalign_train['prompt'] = hallu_factalign_train['prompt'].apply(extract_question)
hallu_factalign_test['prompt'] = hallu_factalign_test['prompt'].apply(extract_question)

In [ ]:
# Concatenate the query and the response, separated by a single space character
def query_answer(row):
  return row['prompt'] + ' ' + row['completion']

hallu_factalign_train['query_answer'] = hallu_factalign_train.apply(query_answer, axis=1)
hallu_factalign_test['query_answer'] = hallu_factalign_test.apply(query_answer, axis=1)

In [ ]:
# Consider only the prompt with the response and the factuality label

hallu_factalign_train = hallu_factalign_train[['query_answer', 'label']]
hallu_factalign_test = hallu_factalign_test[['query_answer', 'label']]

In [ ]:
# Concatenate the train and test splits
hallu_factalign = pd.concat([hallu_factalign_train, hallu_factalign_test], axis=0).reset_index(drop=True)

# Using a random seed for consistency, build again new splits for training, test and validation sets

random.seed(777)

hallu_factalign_train, hallu_factalign_test = train_test_split(hallu_factalign, test_size=0.2, random_state=42)
hallu_factalign_train, hallu_factalign_val = train_test_split(hallu_factalign_train, test_size=0.2, random_state=42)

In [ ]:
# Reconvert the dataset in DatasetDict format, with the new splits (random seed for shuffling)

random.seed(777)

hallu_factalign = DatasetDict({
    'train': Dataset.from_pandas(hallu_factalign_train).shuffle(42),
    'validation': Dataset.from_pandas(hallu_factalign_val).shuffle(42),
    'test': Dataset.from_pandas(hallu_factalign_test).shuffle(42)
})

hallu_factalign

DatasetDict({
    train: Dataset({
        features: ['query_answer', 'label', '__index_level_0__'],
        num_rows: 1639
    })
    validation: Dataset({
        features: ['query_answer', 'label', '__index_level_0__'],
        num_rows: 410
    })
    test: Dataset({
        features: ['query_answer', 'label', '__index_level_0__'],
        num_rows: 513
    })
})

In [ ]:
hallu_factalign['train'][0], hallu_factalign['validation'][0], hallu_factalign['test'][0] # Print some examples for each splits

({'query_answer': "What is the Red Bull Stratos Jump? The Red Bull Stratos Jump was a successful human-powered spaceflight that took place on June 22, 2012, at the New Mexico Test Site in the United States. The mission was led by Felix Baumgartner, a professional skydiver and Red Bull athlete, and was designed to break the sound barrier and reach sub-orbital space.\n\nHere are some specific details and examples related to the Red Bull Stratos Jump:\n\n* Felix Baumgartner, a professional skydiver and Red Bull athlete, was the pilot of the Red Bull Stratos Jump. Baumgartner had previously achieved the highest skydive in the world, which was a tandem skydive from a height of 39,000 feet.\n* The Red Bull Stratos Jump was a human-powered spaceflight, which means that Baumgartner was the sole occupant of the balloon and capsule. The balloon was designed to carry Baumgartner to the edge of space and to allow him to freefall towards the Earth.\n* The Red Bull Stratos Jump was a controlled free

## **Load the model (Roberta)**

In [ ]:
# Fix the number of labels

num_labels = len(set(hallu_factalign['train']['label']))

# Retrieve the model and the corresponding tokenizer from HuggingFace with the method AutoModelForSequenceClassification, to add a classification head at the end of the model,
# in order to perform binary classification using the textual representations generated with the model

model_id = 'FacebookAI/roberta-base'

model = AutoModelForSequenceClassification.from_pretrained(model_id,
                                             num_labels = num_labels)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## **Preprocessing**

Preprocessing phase of the query+response that has to be evaluated as factual or hallucinated.

In [ ]:
max_len = 512
col_to_delete = ['__index_level_0__']

def preprocessing(example):
  return tokenizer(example['query_answer'], truncation=True, max_length=max_len, padding=True, return_tensors='pt')

tokenized_hallu_train = hallu_factalign['train'].map(preprocessing, batched=True, remove_columns=col_to_delete)
tokenized_hallu_val = hallu_factalign['validation'].map(preprocessing, batched=True, remove_columns=col_to_delete)
tokenized_hallu_test = hallu_factalign['test'].map(preprocessing, batched=True, remove_columns=col_to_delete)

Map:   0%|          | 0/1639 [00:00<?, ? examples/s]

Map:   0%|          | 0/410 [00:00<?, ? examples/s]

Map:   0%|          | 0/513 [00:00<?, ? examples/s]

In [ ]:
tokenized_hallu_test

Dataset({
    features: ['query_answer', 'label', 'input_ids', 'attention_mask'],
    num_rows: 513
})

In [ ]:
# Convert everything in torch format for the training
tokenized_hallu_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_hallu_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_hallu_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

## **Training**

In [ ]:
# Build the function with the metrics to be computed during training

def compute_metrics(eval_pred):
    logits, labels = eval_pred # Save the logits of the labels
    predictions = np.argmax(logits, axis=-1) # Consider the label with the highest logit

# Compute the metrics with the predictions and the true labels

    metrics = {
      "accuracy": accuracy_score(y_true=labels, y_pred = predictions),
      "precision": precision_score(y_true=labels, y_pred=predictions, average="binary"),
      "recall": recall_score(y_true=labels, y_pred=predictions, average="binary"),
      "macro_f1": f1_score(y_true=labels, y_pred=predictions, average="binary"),
      "micro_f1": f1_score(y_true=labels, y_pred=predictions, average="binary")
  }
    return metrics

Definition of the parameters for the training. The learning rate has been tried with higher values, but the algorithm did not converge.

In [ ]:
training_args = TrainingArguments(
    output_dir = 'roberta_factalign',
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 5,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    report_to = "none"
)

Set the trainer with the parameters previously defined.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_hallu_train,
    eval_dataset=tokenized_hallu_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
# Start the training

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Macro F1,Micro F1
1,No log,0.491116,0.763415,0.779904,0.761682,0.770686,0.770686
2,No log,0.501836,0.782927,0.761506,0.850467,0.803532,0.803532
3,0.488700,0.707789,0.760976,0.862500,0.644860,0.737968,0.737968
4,0.488700,0.829155,0.795122,0.821782,0.775701,0.798077,0.798077
5,0.230600,0.903109,0.804878,0.813084,0.813084,0.813084,0.813084


TrainOutput(global_step=1025, training_loss=0.35522138595581054, metrics={'train_runtime': 873.423, 'train_samples_per_second': 9.383, 'train_steps_per_second': 1.174, 'total_flos': 2156195098675200.0, 'train_loss': 0.35522138595581054, 'epoch': 5.0})

## **Evaluation**

Evaluate the model on the test set.

In [ ]:
evaluation = trainer.evaluate(tokenized_hallu_test)

## **Saving dataframes with metrics**

Convert the results in a dataframe and save it on Google Drive.

In [ ]:
path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/Results'

In [ ]:
metrics = {
    'classification type' : 'roberta',
    'accuracy' : [evaluation['eval_accuracy']],
    'precision' : [evaluation['eval_precision']],
    'recall' : [evaluation['eval_recall']],
    'macro_f1' : [evaluation['eval_macro_f1']]
}

In [ ]:
metrics

{'classification type': 'roberta',
 'accuracy': [0.7660818713450293],
 'precision': [0.7531914893617021],
 'recall': [0.7405857740585774],
 'macro_f1': [0.7468354430379747]}

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df

,classification type,accuracy,precision,recall,macro_f1
0,roberta,0.766082,0.753191,0.740586,0.746835


In [ ]:
with open(path + "/metrics_roberta_factalign_query_answer.csv", "w") as f:
    metrics_df.to_csv(f, index=False)